<a href="https://colab.research.google.com/github/lfendy/goodcodebadcode/blob/master/goodcodebadcode_zenab_fastai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup with files already in Gdrive

Mounting gdrive to persist data in Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/Shared drives/Cash Eng /ML Infrastructure/05 passion projects/goodcodebadcode/"
base_dir = root_dir + 'fastai-v3/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


###Setting up fastai in the active colab session and importing useful libraries

In [2]:
 !curl -s https://course.fast.ai/setup/colab | bash

Updating fastai...
Done.


In [3]:
from fastai.text import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

### Data Transformation

In [6]:
# loading the parquet file into a dataframe
path = "/content/gdrive/Shared drives/Cash Eng /ML Infrastructure/05 passion projects/goodcodebadcode/data"
%cd /content/gdrive/Shared drives/Cash Eng /ML Infrastructure/05 passion projects/goodcodebadcode/data
df = pd.read_parquet('gcbc_data_all.parquet.gzip')

/content/gdrive/Shared drives/Cash Eng /ML Infrastructure/05 passion projects/goodcodebadcode/data


In [7]:
df.rename(columns = {"Language": "label", "Content": "text"}, inplace = True)
df

,label,text
0,typescript,/**\n * Internal dependencies\n */\nimport con...
1,typescript,import { produce } from 'immer';\n\nimport { m...
2,typescript,/**\n * External dependencies\n */\nimport typ...
3,typescript,/**\n * External Dependencies\n */\nimport { R...
4,typescript,"// Local files\nimport { TitleResponseData, Ca..."
...,...,...
126919,ruby,require 'cocoapods/user_interface/error_report...
126920,ruby,require 'active_support/core_ext/hash/indiffer...
126921,ruby,require 'active_support/core_ext/array'\nrequi...
126922,ruby,module Pod\n class Resolver\n # A small co...


In [8]:
from sklearn.model_selection import train_test_split
train_df, valid_df = train_test_split(df, test_size=0.2)

In [ ]:
# Language model data
data_lm = TextLMDataBunch.from_df(path, train_df = train_df, valid_df = valid_df, test_df = None, label_cols = 0, text_cols = ["text"], no_check = True)

In [20]:
# Classifier model data
data_clas = TextClasDataBunch.from_df(path, train_df=train_df, valid_df=valid_df, vocab=data_lm.train_ds.vocab, bs=32)

Since this step can be a bit time-consuming, it's best to save the result with below command, this will create a 'tmp' directory where all the computed stuff will be stored.

In [22]:
data_lm.save('data_lm_export.pkl')
data_clas.save('data_clas_export.pkl')

We can then reload those results with:

In [10]:
data_lm = load_data(path, 'data_lm_export.pkl')
data_clas = load_data(path, 'data_clas_export.pkl', bs=16)

### Fine-tuning a language model

In [24]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,2.844798,2.333905,0.554241,1:55:35


In [ ]:
learn_code = language_model_learner(data_lm, drop_mult=0.5)
learn_code.fit_one_cycle(1, 1e-2)

In [25]:
learn.unfreeze()
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,1.796419,1.588605,0.686641,2:11:47


To evaluate the language model, we can run the Learner.predict method and specify the number of words we want the model to guess.

In [28]:
learn.predict("fun sum(", n_words=10)

'fun sum(a: Int, b : Int ) { \n  this.id = i'

Note that it understands the vocab which comes from the pretrained language model.

Finally we save the encoder to be able to use it for classification in the next section.

In [29]:
learn.save_encoder('ft_enc')

### Building a classifier

We now use the data_clas object we created earlier to build a classifier with our fine-tuned encoder. The learner object can be done in a single line.

In [ ]:
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5)
learn.load_encoder('ft_enc')

In [31]:
data_clas.show_batch()

text,target
xxbos # ! / usr / bin / env node \n module.exports = / xxrep 6 * / ( function ( modules ) { \n \t / / xxunk \n \t / xxrep 6 * / / / xxmaj the module cache \n \t / xxrep 6 * / var installedmodules = { } ; / / xxmaj the require function \n \t / xxrep,javascript
xxbos / * * \n * xxmaj licensed to the xxmaj apache xxmaj software xxmaj foundation ( xxup asf ) under one \n * or more contributor license agreements . xxmaj see the xxup notice file \n * distributed with this work for additional information \n * regarding copyright ownership . xxmaj the xxup asf licenses this file \n * to you under the xxmaj,java
"xxbos require xxmaj file.expand_path ( ' .. / .. / .. / spec_helper ' , _ _ xxup file _ _ ) \n \n module xxmaj pod \n describe xxunk do # rubocop : disable xxmaj metrics / xxunk \n describe ' xxmaj analysis ' do # rubocop : disable xxmaj metrics / xxunk \n before do \n repos = [ xxmaj xxunk - repos",ruby
"xxbos / * ! jquery xxunk | ( c ) 2005 , 2014 jquery xxmaj foundation , xxmaj inc. | jquery.org / license * / \n ! function(a , xxunk xxunk xxunk new xxunk requires a window with a xxunk xxunk xxunk : this , function(a , b){var xxunk = xxunk , e = xxunk , f = xxunk , g = xxunk , xxunk = xxunk , j",javascript
"xxbos # -*- coding : utf-8 -*- # \n # frozen_string_literal : true \n \n module xxmaj rouge \n module xxmaj lexers \n class xxup xxunk \n def xxunk \n @commands ||= xxmaj set.new [ "" abs "" , "" acos "" , "" actionids "" , "" xxunk "" , "" xxunk "" , "" xxunk "" , "" xxunk "" , """,ruby


In [32]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.036638,0.023630,0.991215,1:53:30


We are unfreezing the last two modules and fine tuning it

In [ ]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

Unfreezing all of it and fine tuning again

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

We can now try predicting using raw data

In [33]:
learn.predict("fun sum(a: Int, b: Int): Int { return a + b }")

(Category tensor(2),
 tensor(2),
 tensor([3.9778e-04, 2.4990e-07, 9.9960e-01, 1.7361e-08, 2.0405e-07]))